In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics

from warnings import simplefilter

import joblib

2024-02-26 08:54:41.730362: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 08:54:41.733108: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 08:54:41.762905: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 08:54:41.762955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 08:54:41.763965: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
#with two dataset splitted
dftrain = pd.read_csv("/home/jovyan/MQTTset/train70_reduced.csv") 
dftest = pd.read_csv("/home/jovyan/MQTTset/test30_reduced.csv")

# dftrain = pd.read_csv("train70.csv", low_memory=False) 
# dftest = pd.read_csv("test30.csv", low_memory=False)

simplefilter(action='ignore', category=FutureWarning)
seed = 7

#train
#print(dftrain.loc[dftrain['target'] == 'legitimate'])
class_names = dftrain.target.unique()
dftrain=dftrain.astype('category')
cat_columns = dftrain.select_dtypes(['category']).columns
dftrain[cat_columns] = dftrain[cat_columns].apply(lambda x: x.cat.codes)
#print(dftrain.loc[125, 'target'])
x_columns = dftrain.columns.drop('target')
x_train = dftrain[x_columns].values
y_train = dftrain['target']

#test
class_names = dftest.target.unique()
dftest=dftest.astype('category')
cat_columns = dftest.select_dtypes(['category']).columns
dftest[cat_columns] = dftest[cat_columns].apply(lambda x: x.cat.codes)
x_columns = dftest.columns.drop('target')
x_test = dftest[x_columns].values
y_test = dftest['target']

print("Ready to generate train and test datasets")

from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

x_train = np.copy(x_train_scaled)
x_test = np.copy(x_test_scaled)

Ready to generate train and test datasets


In [4]:
model = joblib.load("/home/jovyan/MQTTset/dl/new_dl/dnn.joblib")

2024-02-26 08:54:47.494690: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://12f53c7fb7b34a72b867fa9473b8de28: INVALID_ARGUMENT: ram://12f53c7fb7b34a72b867fa9473b8de28 is a directory.


In [5]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5], dtype=int8)

In [6]:
x_train.shape

(231646, 33)

In [7]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=6,
    input_shape=(33,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [22]:
attack = FastGradientMethod(estimator=classifier, eps=0.01)

x_test_adv = attack.generate(x=x_test)

In [23]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 650us/step
Accuracy: 0.06002618592003223
AUC: 0.5961863165227622

macro
Precision: 0.8389884895660815
Recall: 0.20656330041322138
F1 Score: 0.07969956012320527

weighted
Precision: 0.9681058073601138
Recall: 0.06002618592003223
F1 Score: 0.05179905604065773

Mean FNR: 0.7934366995867785
Mean TNR: 0.8379890013748281
Mean FPR: 0.16201099862517185
Mean TPR: 0.20656330041322138


In [24]:
attack = FastGradientMethod(estimator=classifier, eps=0.05)

x_test_adv = attack.generate(x=x_test)

In [25]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 604us/step
Accuracy: 0.22085809245644072
AUC: 0.6495704728073199

macro
Precision: 0.7722276713749526
Recall: 0.26414558078227984
F1 Score: 0.15426755166614733

weighted
Precision: 0.8064590951771922
Recall: 0.22085809245644072
F1 Score: 0.22058000830721092

Mean FNR: 0.7358544192177202
Mean TNR: 0.8526818337652933
Mean FPR: 0.14731816623470675
Mean TPR: 0.26414558078227984


In [12]:
attack = FastGradientMethod(estimator=classifier, eps=0.1)

x_test_adv = attack.generate(x=x_test)

In [13]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 635us/step
Accuracy: 0.40187330043307484
AUC: 0.7030502691402979

macro
Precision: 0.5879672274028883
Recall: 0.3133292578862421
F1 Score: 0.2414812973587944

weighted
Precision: 0.7407642819405884
Recall: 0.40187330043307484
F1 Score: 0.4174014675195846

Mean FNR: 0.6866707421137579
Mean TNR: 0.8735632110769197
Mean FPR: 0.12643678892308033
Mean TPR: 0.3133292578862421


In [14]:
attack = FastGradientMethod(estimator=classifier, eps=0.15)

x_test_adv = attack.generate(x=x_test)

In [15]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 591us/step
Accuracy: 0.4944002417161849
AUC: 0.6182828313260625

macro
Precision: 0.5207644248473271
Recall: 0.330745023877368
F1 Score: 0.26719381984638646

weighted
Precision: 0.7864287036305286
Recall: 0.4944002417161849
F1 Score: 0.5647480537527988

Mean FNR: 0.6692549761226321
Mean TNR: 0.9005037589744659
Mean FPR: 0.09949624102553416
Mean TPR: 0.330745023877368


In [16]:
attack = FastGradientMethod(estimator=classifier, eps=0.2)

x_test_adv = attack.generate(x=x_test)

In [17]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 619us/step
Accuracy: 0.4712357739953671
AUC: 0.5663366590169687

macro
Precision: 0.6979182086736251
Recall: 0.3215742178453354
F1 Score: 0.26514780839215063

weighted
Precision: 0.844720249526362
Recall: 0.4712357739953671
F1 Score: 0.5628866213391991

Mean FNR: 0.6784257821546648
Mean TNR: 0.901024703429041
Mean FPR: 0.09897529657095895
Mean TPR: 0.3215742178453354


In [18]:
attack = FastGradientMethod(estimator=classifier, eps=0.25)

x_test_adv = attack.generate(x=x_test)

In [19]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 619us/step
Accuracy: 0.4619800584147447
AUC: 0.5506317547926011

macro
Precision: 0.3638329017970643
Recall: 0.3176500685963209
F1 Score: 0.2630745671594673

weighted
Precision: 0.8062673204740781
Recall: 0.4619800584147447
F1 Score: 0.5580897855541619

Mean FNR: 0.682349931403679
Mean TNR: 0.89970461601088
Mean FPR: 0.10029538398912007
Mean TPR: 0.3176500685963209


In [20]:
attack = FastGradientMethod(estimator=classifier, eps=0.3)

x_test_adv = attack.generate(x=x_test)

In [21]:
calculate_performance_metrics(x_test_adv, y_test, model)

3103/3103 [==============================] - 2s 613us/step
Accuracy: 0.4160338402658878
AUC: 0.545100077280249

macro
Precision: 0.33158826862732627
Recall: 0.298192757816749
F1 Score: 0.24315899095695093

weighted
Precision: 0.7131222816607711
Recall: 0.4160338402658878
F1 Score: 0.5080195922219304

Mean FNR: 0.701807242183251
Mean TNR: 0.8853505811867467
Mean FPR: 0.1146494188132533
Mean TPR: 0.298192757816749
